# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [152]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv
from panel.interact import interact
from panel import widgets

import warnings
warnings.filterwarnings('ignore')

In [158]:
# Read the Mapbox API key
load_dotenv('example.env')
map_box_api = os.getenv("MAPBOX_API_KEY")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [66]:
# Import the necessary CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")
sfo_data.head()

sfo_avg_data_year = sfo_data.groupby('year').mean() # Average Data computed by Year
sfo_avg_neighbrohood_df = sfo_data.groupby(by=['year', 'neighborhood']).mean().reset_index() # Average Data by Neighborhood and Year

neighborhood_coordinate_path = Path("Data/neighborhoods_coordinates.csv")
neighbor_cord_df = pd.read_csv(neighborhood_coordinate_path)
neighbor_cord_df.head()

,Neighborhood,Lat,Lon
0,Alamo Square,37.791012,-122.402100
1,Anza Vista,37.779598,-122.443451
2,Bayview,37.734670,-122.401060
3,Bayview Heights,37.728740,-122.410980
4,Bernal Heights,37.728630,-122.443050


- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [150]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    # Average Data by year computed above is used here
    
    minimum = sfo_avg_data_year['housing_units'].min() - sfo_avg_data_year['housing_units'].std()
    maximum = sfo_avg_data_year['housing_units'].max() + sfo_avg_data_year['housing_units'].std()
    plot = sfo_avg_data_year.hvplot.bar(x='year', y='housing_units', title='Housing Units per Year', xlabel='Year', ylabel='Housing Units',  ylim =(minimum,maximum))
    
    return plot

def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    # Average Data by year computed above is used here
       
    plot = sfo_avg_data_year['gross_rent'].hvplot(x='year', y='gross_rent', title = 'Avg Gross Rent by Year', label='Year', ylabel='Gross Rent')
    
    return plot

def average_sales_price():
    """Average Sales Price Per Year."""
    # Average Data by year computed above is used here
    
    plot = sfo_avg_data_year['sale_price_sqr_foot'].hvplot(x='year', y='sale_price_sqr_foot', title = 'Aerage Sales Price per Sq Ft', label='Year', ylabel='Avg Sales Price')
    
    return plot


def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
     # Average Data by year & neighborhoo computed above: sfo_avg_neighbrohood_df is used here
    
    neighborhoods = sfo_avg_neighbrohood_df.neighborhood.unique() # Get the unique list of neighborhoods
    
    # Define function to create plot
    def plot_avg_price(neighborhood):
        df = sfo_avg_neighbrohood_df.loc[sfo_avg_neighbrohood_df['neighborhood'] == neighborhood ]
        
        return df.hvplot.line(
            x = "year",
            y = "sale_price_sqr_foot",
            title="Average price per sq ft", 
            )
    return interact(plot_avg_price, neighborhood = neighborhoods)

def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    # Average Data by year & neighborhoo computed above: sfo_avg_neighbrohood_df is used here
    
    sfo_df_top_10 = sfo_avg_neighbrohood_df.groupby('neighborhood').mean().sort_values(by='sale_price_sqr_foot', ascending=False )
    sfo_df_top_10 = sfo_df_top_10.reset_index().head(10)
        
    plot = sfo_df_top_10.hvplot.bar(x='neighborhood', y ='sale_price_sqr_foot', title = 'Sale price/Sq.Ft by Neighborhood', rot = 90,
                                    width=800, height=400, xlabel = 'Neighborhood', ylabel = 'Sale Price')

    return plot


def most_expensive_neighborhoods_rent_sales():
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""
    # Average Data by year & neighborhoo computed above: sfo_avg_neighbrohood_df is used here
    
    sfo_df_top_10 = sfo_avg_neighbrohood_df.groupby('neighborhood').mean().sort_values(by='sale_price_sqr_foot', ascending=False )
    sfo_df_top_10 = sfo_df_top_10.reset_index().head(10)
    neighborhoods_top_10 = sfo_df_top_10.neighborhood.unique()

    def plot_avg_price(neighborhood):
        df = sfo_avg_neighbrohood_df.loc[sfo_avg_neighbrohood_df['neighborhood'] == neighborhood ]

        return df.hvplot.line(
            x = "year", y = "sale_price_sqr_foot", title="Sale Price per Sq Ft", width=800, height=400, 
            xlabel = 'Year', ylabel = 'Sale Price'
            )

    def plot_gross_rent(neighborhood):
        df = sfo_avg_neighbrohood_df.loc[sfo_avg_neighbrohood_df['neighborhood'] == neighborhood ]

        return df.hvplot.line(
            x = "year", y = "gross_rent", title="Average Monthly Rent", width=800, height=400, 
            xlabel = 'Year', ylabel = 'Gross Rent'
            )

    # Render plot with Panel interactive widget
    return pn.Row(interact(plot_avg_price, neighborhood = neighborhoods_top_10), interact(plot_gross_rent, neighborhood = neighborhoods_top_10))
    
    
    
def parallel_coordinates():
    """Parallel Coordinates Plot."""
    # Average Data by year & neighborhoo computed above: sfo_avg_neighbrohood_df is used here
        
    sfo_df_top_10 = sfo_avg_neighbrohood_df.groupby('neighborhood').mean().sort_values(by='sale_price_sqr_foot', ascending=False )
    sfo_df_top_10 = sfo_df_top_10.reset_index().head(10)
    neighborhoods_top_10 = sfo_df_top_10.neighborhood.unique()
    
    df_expensive_neighborhoods_per_year = sfo_avg_neighbrohood_df[sfo_avg_neighbrohood_df["neighborhood"].isin(neighborhoods_top_10)]
    df_top_10 = df_expensive_neighborhoods_per_year.groupby('neighborhood').mean().reset_index().sort_values("sale_price_sqr_foot", ascending=False)
    
    parallel_plot = px.parallel_coordinates(
        df_top_10,
        dimensions=["neighborhood","sale_price_sqr_foot", "housing_units", "gross_rent"],
        color="sale_price_sqr_foot",
        color_continuous_scale=px.colors.sequential.Inferno,
        labels={
            "neighborhood": "Neighborhood",
            "sale_price_sqr_foot" : "Sale Price Sq Ft",
            "housing_units": "Housing_units",
            "gross_rent": "Gross_rent"
        },
        width = 800
        )
    return parallel_plot
    

def parallel_categories():
    """Parallel Categories Plot."""
    # Average Data by year & neighborhoo computed above: sfo_avg_neighbrohood_df is used here
        
    sfo_df_top_10 = sfo_avg_neighbrohood_df.groupby('neighborhood').mean().sort_values(by='sale_price_sqr_foot', ascending=False )
    sfo_df_top_10 = sfo_df_top_10.reset_index().head(10)
    neighborhoods_top_10 = sfo_df_top_10.neighborhood.unique()
    
    df_expensive_neighborhoods_per_year = sfo_avg_neighbrohood_df[sfo_avg_neighbrohood_df["neighborhood"].isin(neighborhoods_top_10)]
    df_top_10 = df_expensive_neighborhoods_per_year.groupby('neighborhood').mean().reset_index().sort_values("sale_price_sqr_foot", ascending=False)

    parallel_plot = px.parallel_categories(
        df_top_10,
        dimensions=["neighborhood","sale_price_sqr_foot", "housing_units", "gross_rent"],
        color="sale_price_sqr_foot",
        color_continuous_scale=px.colors.sequential.Inferno,
        labels={
            "neighborhood": "Neighborhood",
            "housing_units": "Housing_units",
            "gross_rent": "Gross_rent"
            },
        width = 800
        )
    
    
    return parallel_plot


px.set_mapbox_access_token(map_box_api)

def neighborhood_map():
    """Neighborhood Map."""
    # Average Data by year & neighborhoo computed above: sfo_avg_neighbrohood_df is used here
    sfo_neighborhood_mean = sfo_avg_neighbrohood_df[['neighborhood', 'sale_price_sqr_foot', 'housing_units', 'gross_rent']].groupby('neighborhood').mean().reset_index()
    sfo_cord_df = neighbor_cord_df.join(sfo_neighborhood_mean.set_index(['neighborhood']), on = 'Neighborhood', how ='inner')
    
    # Create a scatter mapbox to analyze neighborhood info
    map_plot = px.scatter_mapbox(
        sfo_cord_df,
        lat="Lat",
        lon="Lon",
        size = "sale_price_sqr_foot", 
        color="gross_rent",
        zoom = +10
    )

#     print(type(map_plot))
    return map_plot


def sunburst():
    """Sunburst Plot."""
    # Average Data by year & neighborhoo computed above: sfo_avg_neighbrohood_df is used here
    sfo_df_neighborhood = sfo_avg_neighbrohood_df[['neighborhood', 'sale_price_sqr_foot', 'housing_units', 'gross_rent']].groupby('neighborhood').mean().sort_values(by='sale_price_sqr_foot', ascending=False )
    sfo_df_neighborhood = sfo_df_neighborhood.reset_index().head(10)
    neighborhoods_top_10 = sfo_df_neighborhood.neighborhood.unique()
    
    df_2 = sfo_avg_neighbrohood_df[sfo_avg_neighbrohood_df["neighborhood"].isin(neighborhoods_top_10)][['year','neighborhood','sale_price_sqr_foot', 'gross_rent']]
    print(df_2.head())
    
    sunburstplot = px.sunburst(
        df_2,
        path=["year","neighborhood"],
        values = "sale_price_sqr_foot",
        color="gross_rent"
        )
    return sunburstplot


## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

# San Francisco Rental Prices Dashboard

In [174]:
# Create a Title for the Dashboard
neighborhood = pn.Column("""This dashboard presents visual anaylsis of huosing units sold, sale price and rent in Sanfrancisco, California
                      from 2010 to 2016. Please neviagte through different tabs of the dashboard.""", neighborhood_map()) 
housing_unit = pn.Row("## Housing Unit per Year", housing_units_per_year(),average_gross_rent(), average_sales_price()) 
average_rent_price = pn.Column('## Average Sales Price by Neighborhood', average_price_by_neighborhood(), top_most_expensive_neighborhoods)
parallel_plot = pn.Column('## Parallel Categories & Coordinates', parallel_categories(), parallel_coordinates())
sunbrust_plot = pn.Column('## SFO Sunburst Plot', average_gross_rent(), average_sales_price(), average_price_by_neighborhood())


# Create a tab layout for the dashboard
# YOUR CODE HERE!

dashboard = pn.Tabs(
    ("Welcome", neighborhood),
    ("Yearly Market Analysis", housing_unit),
    ("Average Rent & Price", average_rent_price),
    ("Parallel Plot Analysis", parallel_plot),
    ("Sunburst Plot Analysis", sunbrust_plot),
)

# Create the dashboard
dashboard

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
    [1] Row
        [0] Markdown(str)
        [1] HoloViews(Bars)
        [2] HoloViews(Curve)
        [3] HoloViews(Curve)
    [2] Column
        [0] Markdown(str)
        [1] Column
            [0] Column
                [0] Select(name='neighborhood', options=['Alamo Square', ...], value='Alamo Square')
            [1] Row
                [0] HoloViews(Curve, name='interactive58192')
        [2] Column
            [0] Column()
            [1] Row
                [0] HoloViews(Bars, name='interactive58282')
    [3] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
        [2] Plotly(Figure)
    [4] Column
        [0] Markdown(str)
        [1] HoloViews(Curve)
        [2] HoloViews(Curve)
        [3] Column
            [0] Column
                [0] Select(name='neighborhood', options=['Alamo Square', ...], value='Alamo Square')
            [1] Row
                [0] HoloViews(Curve, name='interactive58525')

## Serve the Panel Dashboard

In [173]:
# Serve the# dashboard
dashboard.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Plotly(Figure, relayout_data={'mapbox.center': {'lon': ...})
    [1] Row
        [0] Markdown(str)
        [1] HoloViews(Bars)
        [2] HoloViews(Curve)
        [3] HoloViews(Curve)
    [2] Column
        [0] Markdown(str)
        [1] Column
            [0] Column
                [0] Select(name='neighborhood', options=['Alamo Square', ...], value='Alamo Square')
            [1] Row
                [0] HoloViews(Curve, name='interactive56082')
        [2] Column
            [0] Column()
            [1] Row
                [0] HoloViews(Bars, name='interactive56172')
    [3] Column
        [0] Markdown(str)
        [1] Plotly(Figure, relayout_data={'autosize': True})
        [2] Plotly(Figure, relayout_data={'autosize': True})
    [4] Column
        [0] Markdown(str)
        [1] HoloViews(Curve)
        [2] HoloViews(Curve)
        [3] Column
            [0] Column
                [0] Select(name='neighborhood', options=['Alamo Square', ...], value='Alamo Square')
            [1] Row
                [0] HoloViews(Curve, name='interactive56415')

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [84]:
# housing_units_per_year()

In [83]:
# average_gross_rent()

In [82]:
# average_sales_price()

In [87]:
# average_price_by_neighborhood()

In [96]:
# top_most_expensive_neighborhoods()

In [106]:
# most_expensive_neighborhoods_rent_sales()

In [160]:
# display(neighborhood_map())

In [111]:
neighborhood_map().show()

<class 'plotly.graph_objs._figure.Figure'>


In [139]:
# parallel_categories()

In [138]:
# parallel_coordinates()

In [141]:
# sunburst()